In [1]:
class Node:
    max_length = 8
    
    def __init__(self, state: str):
        self.state = state

    def __repr__(self):
        return self.state

    def __hash__(self):
        return hash(self.state)

    def dup(self) -> Node:
        if len(self.state) < Node.max_length:
            return Node(self.state + self.state[-1])

    def swap(self) -> Node:
        if len(self.state) > 1:
            return Node(self.state[:-2] + self.state[-1] + self.state[-2])

    def rot(self) -> Node:
        if len(self.state) > 2:
            return Node(self.state[:-3] + self.state[-1] + self.state[-2] + self.state[-3])

    def pop(self) -> Node:
        if len(self.state) > 2:
            return Node(self.state[:-1])

    transition_rules = {
        dup: False,
        swap: True,
        rot: True,
    }

In [2]:
from itertools import product


class Graph:
    def __init__(self, alphabet: str):
        self.alphabet = alphabet
        self.nodes = set()
        self.edges = list()
        
    def compute_all_nodes(self):
        self.nodes = [
            Node(''.join(permutation))
            for n in range (1, Node.max_length+1)
            for permutation in product(self.alphabet, repeat = n)
        ]

    def compute_all_edges(self):
        for source_node in self.nodes:
            for rule in Node.transition_rules:
                target_node = rule(source_node)
                if target_node is not None:
                    self.edges.append((
                        source_node,
                        target_node,
                        rule
                    ))

In [3]:
g = Graph("abc")
g.compute_all_nodes()
g.compute_all_edges()

In [5]:
import pandas as pd
from cosmograph import cosmo
import ipywidgets as widgets

colors = {
    Node.dup: '#fa2020',
    Node.swap: '#2070fa',
    Node.rot: '#46db1b',
}

nodes = pd.DataFrame({
    'label': [node.state for node in g.nodes],
})

edges = pd.DataFrame({
    'source': [edge[0].state for edge in g.edges],
    'target': [edge[1].state for edge in g.edges],
    'arrow': [not Node.transition_rules[edge[2]] for edge in g.edges],
    'color': [colors[edge[2]] for edge in g.edges]
})

widget = cosmo(
    points=nodes,
    links=edges,
    
    point_id_by='label',
    link_source_by='source',
    link_target_by='target',
    point_include_columns=['value'],
    point_label_by='label',
    link_include_columns=['value'],

    link_arrows= True,
    link_width = 1.5,
    link_arrow_by='arrow',
    link_color_by='color',
    space_size =2000,
    point_color = '#ccc',
    point_size = 2,
    
    simulation_link_spring=3,
    simulation_link_distance=4,
    link_arrows_size_scale=2,
    simulation_gravity=0.3,
    simulation_repulsion=1.7,
    simulation_decay=2000,
    background_color='black',
    show_labels=True
)

widget.layout.height = '800px'

widget